In [16]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import csv
from pandas.tseries.offsets import MonthEnd
from datetime import datetime, timedelta, date
import requests
import json 
import ta




# READ CSV 

In [76]:
#data with feature engineering
fields = ['Date', 'Open','High','Low','Close','Volume','RSI','MACD','Ichimoku_Span_A','Ichimoku_Span_B','ATR',
         'BB_High_Indicator','BB_Low_Indicator']
MSFT_df = pd.read_csv(r"/Users/fahad/project_repo/data/interim/MSFT.csv", usecols=fields)
AAPL_df = pd.read_csv(r"/Users/fahad/project_repo/data/interim/AAPL.csv", usecols=fields)
AMZN_df = pd.read_csv(r"/Users/fahad/project_repo/data/interim/AMZN.csv", usecols=fields)


,Date,Open,High,Low,Close,Volume,RSI,MACD,Ichimoku_Span_A,Ichimoku_Span_B,ATR,BB_High_Indicator,BB_Low_Indicator
0,2019-07-10,0.744656,0.746298,0.759324,0.742976,0.016024,0.517256,0.325001,0.231939,0.236938,0.125655,0.0,0.0
1,2019-07-09,0.737755,0.736526,0.751678,0.736149,0.027496,0.000000,0.324198,0.231939,0.236938,0.127104,0.0,0.0
2,2019-07-08,0.740198,0.737315,0.749006,0.739183,0.022672,0.350682,0.324890,0.231939,0.236938,0.128062,0.0,0.0
3,2019-07-05,0.737389,0.738711,0.751181,0.739790,0.024988,0.400295,0.325302,0.231939,0.236938,0.128110,0.0,0.0
4,2019-07-03,0.742641,0.741199,0.754755,0.742217,0.017310,0.559390,0.326239,0.231939,0.236938,0.126996,0.0,0.0


# GET THE MONTH ENDINGS

In [5]:
#ensure the the data frames have been loaded correctly 
# print (MSFT_df.head(5))
# print ("\n")
# print (AAPL_df.head(5))
# print ("\n")
# print (AMZN_df.head(5))



In [6]:
#create second sent of dataframes with only the open high low close and volume
#data without feature engineering
fields = ['Date', 'Open','High','Low','Close','Volume']
MSFT_df_no_fe = pd.read_csv(r"/Users/fahad/project_repo/data/interim/MSFT_no_fe.csv", usecols=fields)
AAPL_df_no_fe = pd.read_csv(r"/Users/fahad/project_repo/data/interim/AAPL_no_fe.csv", usecols=fields)
AMZN_df_no_fe = pd.read_csv(r"/Users/fahad/project_repo/data/interim/AMZN_no_fe.csv", usecols=fields)

# Call Monthly OHLC to create the labels for the classification task

In [19]:
#dict for the month endings which we will create a dataframe with the close prices from 
#apikey=539COEUG1C1IYXMF
MSFT = requests.get("https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=MSFT&outputsize=full&apikey=539COEUG1C1IYXMF")
MSFT = MSFT.json() 
AMZN = requests.get("https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=AMZN&outputsize=full&apikey=539COEUG1C1IYXMF")
AMZN = AMZN.json()
AAPL = requests.get("https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=AAPL&outputsize=full&apikey=539COEUG1C1IYXMF")
AAPL = AAPL.json()

In [20]:
#transform the json object into a pandas dataframe object and transpose
MSFT_df_monthly = pd.DataFrame(MSFT['Monthly Time Series'])
MSFT_df_monthly = MSFT_df_monthly.T
#transform the json object into a pandas dataframe object and transpose
AMZN_df_monthly = pd.DataFrame(AMZN['Monthly Time Series'])
AMZN_df_monthly = AMZN_df_monthly.T
#transform the json object into a pandas dataframe object and transpose
AAPL_df_monthly = pd.DataFrame(AAPL['Monthly Time Series'])
AAPL_df_monthly = AAPL_df_monthly.T

print (MSFT_df_monthly.head(5))
print ("\n")
print (AAPL_df_monthly.head(5))
print ("\n")
print (AMZN_df_monthly.head(5))
print ("\n")

             1. open   2. high    3. low  4. close  5. volume
2019-08-09  137.0000  140.9400  130.7800  137.7100  229623586
2019-07-31  136.6300  141.6800  134.6700  136.2700  484547500
2019-06-28  123.8500  138.4000  119.0100  133.9600  508324300
2019-05-31  130.5300  130.6500  123.0400  123.6800  547218800
2019-04-30  118.9500  131.3700  118.1000  130.6000  433157700


             1. open   2. high    3. low  4. close  5. volume
2019-08-09  213.9000  218.0300  192.5800  200.9900  266981630
2019-07-31  203.1700  221.3700  198.4100  213.0400  473957000
2019-06-28  175.6000  201.5700  170.2700  197.9200  515218700
2019-05-31  209.8800  215.3100  174.9900  175.0700  739456600
2019-04-30  191.6400  208.4800  188.3800  200.6700  506117700


              1. open    2. high     3. low   4. close 5. volume
2019-08-09  1871.7200  1897.9200  1748.7800  1807.5800  31578820
2019-07-31  1922.9800  2035.8000  1849.4399  1866.7800  73177400
2019-06-28  1760.0100  1935.2000  1672.0000  1893.6300  7

# Format the DataFrame Objects

In [21]:
MSFT_df_monthly.index.name = "Date"
MSFT_df_monthly.columns = ['Open', 'High','Low','Close','Volume']

#drop the last row of each dataframe so the entries match
#MSFT_df_monthly.drop(MSFT_df_monthly.tail(1).index,inplace=True)

AAPL_df_monthly.index.name="Date"
AAPL_df_monthly.columns =["Open","High","Low","Close","Volume"]
#AAPL_df_monthly.drop(AAPL_df_monthly.tail(1).index,inplace=True)

AMZN_df_monthly.index.name = "Date"
AMZN_df_monthly.columns = ["Open","High","Low","Close","Volume"]
#AMZN_df_monthly.drop(AMZN_df_monthly.tail(1).index,inplace=True)

print (MSFT_df_monthly.head(5))
print (AAPL_df_monthly.head(5))
print (AMZN_df_monthly.head(5))

                Open      High       Low     Close     Volume
Date                                                         
2019-08-09  137.0000  140.9400  130.7800  137.7100  229623586
2019-07-31  136.6300  141.6800  134.6700  136.2700  484547500
2019-06-28  123.8500  138.4000  119.0100  133.9600  508324300
2019-05-31  130.5300  130.6500  123.0400  123.6800  547218800
2019-04-30  118.9500  131.3700  118.1000  130.6000  433157700
                Open      High       Low     Close     Volume
Date                                                         
2019-08-09  213.9000  218.0300  192.5800  200.9900  266981630
2019-07-31  203.1700  221.3700  198.4100  213.0400  473957000
2019-06-28  175.6000  201.5700  170.2700  197.9200  515218700
2019-05-31  209.8800  215.3100  174.9900  175.0700  739456600
2019-04-30  191.6400  208.4800  188.3800  200.6700  506117700
                 Open       High        Low      Close    Volume
Date                                                            
20

# Create Labels

In [55]:
#empty dictionaries to store the labels
# MSFT_labels = dict()

# AAPL_labels = dict()

# AMZN_labels = dict()

#reversed monthly Dataframe 
reversed_df_MSFT = MSFT_df_monthly.iloc[::-1]
reversed_df_AAPL= AAPL_df_monthly.iloc[::-1]
reversed_df_AMZN = AMZN_df_monthly.iloc[::-1]

reversed_df_MSFT_multiclass= MSFT_df_monthly.iloc[::-1]
reversed_df_AAPL_multiclass= AAPL_df_monthly.iloc[::-1]
reversed_df_AMZN_multiclass = AMZN_df_monthly.iloc[::-1]

def labelling(dataframe):
    """This generates labels for the dataframes we wish to predict price.
    A label of 1 suggests an increase in price between months while a label of 0 indicates
    a decrease in price. If the current price is greater than the previous price then a"""
    empty_dict = dict()
    prev_close =0
    counter = 0
    for index,cols in dataframe.iterrows():
        counter +=1
        current_close = cols['Close']
        if float(prev_close) <= float(current_close):
            #signals an increase in price between last month and next month
            empty_dict[index] = 1
        elif float(prev_close) >= float(current_close):
            empty_dict[index]=0

        #reassign prev close to the current close     
        prev_close = current_close
    
    return empty_dict

#ADD LABELS TO THE REVERESED DATAFRAMES USING THE LABELLING FUNCTION 
MSFT_labels = labelling(reversed_df_MSFT)
AAPL_labels = labelling(reversed_df_AAPL)
AMZN_labels = labelling(reversed_df_AMZN)

# print (list(MSFT_labels.values())[:20])
# print (list(AAPL_labels.values())[:20])
# print (list(AMZN_labels.values())[:20])

len(MSFT_labels.values()) 

240

In [14]:
#Add the labels to the reversed DataFrames
reversed_df_MSFT['Labels'] = MSFT_labels.values()
reversed_df_AAPL['Labels'] = AAPL_labels.values()
reversed_df_AMZN['Labels']= AMZN_labels.values()


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [15]:
reversed_df_MSFT.index[-1]

'2019-07-23'

In [25]:
reversed_df_MSFT.head(1)

,Open,High,Low,Close,Volume,Labels
Date,,,,,,
1998-02-27,151.7500,160.0600,79.3700,84.7500,215523150,1


# ADD LABELS TO THE FEATURE ENGINEERED CSV/DATAFRAMES

# Get previous 30 entries from the month endings 

In [77]:
def get_ndays_monthly(dataframe_period_end,dataframe_subperiod,n):
    """returns a dictionary which contains key value pair of the month-end date as key and
    value is the proceeding n days. Returns it as a dictionary of dataframes.
    dataframe period end refers to the REVERESED dataframe which will act as the period end i.e monthly end date
    sub_period dataframe refers to the dataframe with the preceeding n days, i.e daily. So for example will have 
    a dataframe of the month ending of february 2019-02-28:30days of January"""
    empty_dict = dict()
    count = 0
    for end in dataframe_period_end.index:
        #print (end)
#         print ("\n")
        
        month_key = dataframe_period_end.index[count+1]
        #print ("month key:%s" % month_key)
        last_index = dataframe_period_end.index[-1]
        #print ("last index:%s" % last_index)
        
        if (month_key != last_index):
#         print ("index at position %s is :%s" % (count,dataframe_period_end.index[count]))     
            temp_df = dataframe_subperiod.loc[dataframe_subperiod['Date']<end].head(n)
#             print ("month key: %s" % month_key)
#             print ("\n")
#             print (temp_df.head(5))
        
            empty_dict[month_key] = temp_df
        
            count +=1
        else:
            pass
       
    return empty_dict



MSFT_30day = get_ndays_monthly(reversed_df_MSFT,MSFT_df,30)
AAPL_30day = get_ndays_monthly(reversed_df_AAPL,AAPL_df,30)
AMZN_30day = get_ndays_monthly(reversed_df_AAPL,AAPL_df,30)

#MSFT_30day = get_ndays_monthly(MSFT_df_monthly,MSFT_df,30)
# AAPL_30day = get_ndays_monthly(AAPL_df_monthly,AAPL_df,30)
# AMZN_30day = get_ndays_monthly(AMZN_df_monthly,AMZN_df,30)





# print (list(MSFT_30day.values())[:1])
# print ("\n")
# print (list(AAPL_30day.values())[:1])
# print ("\n")
# print (list(AMZN_30day.values())[:1])



In [88]:
MSFT_30day


{'1999-10-29':             Date      Open      High       Low     Close    Volume       RSI  \
 4975  1999-09-29  0.466349  0.464312  0.461524  0.451180  0.026011  0.392614   
 4976  1999-09-28  0.464456  0.467771  0.461151  0.467079  0.033258  0.551028   
 4977  1999-09-27  0.469036  0.469289  0.472402  0.462953  0.023159  0.513106   
 4978  1999-09-24  0.457982  0.459760  0.459970  0.459919  0.053915  0.484800   
 4979  1999-09-23  0.498779  0.493142  0.467118  0.461436  0.054593  0.501756   
 4980  1999-09-22  0.485831  0.491624  0.489930  0.490989  0.035412  0.719888   
 4981  1999-09-21  0.496885  0.491260  0.492603  0.482250  0.034179  0.631651   
 4982  1999-09-20  0.493465  0.499211  0.498073  0.500091  0.035471  0.721913   
 4983  1999-09-17  0.483510  0.490896  0.490676  0.493294  0.061996  0.657190   
 4984  1999-09-16  0.474350  0.476086  0.470848  0.478488  0.033084  0.536320   
 4985  1999-09-15  0.490412  0.486344  0.481726  0.470113  0.035311  0.478847   
 4986  1999-09

In [63]:
def get_ndays_monthly_flattened(dataframe_period_end,dataframe_subperiod,n):
    """returns a dictionary which contains key value pair of the month-end date as key and
    value is the proceeding n days. Returns it as a dictionary of dataframes.
    dataframe period end refers to the dataframe which will act as the period end i.e monthly end date
    sub_period dataframe refers to the dataframe with the preceeding n days, i.e daily"""
    empty_dict = dict()
    count = 0
    for end in dataframe_period_end.index:
#         print ("month end/:%s" % end)
        
        month_key = dataframe_period_end.index[count+1]
        
#         print ("month key:%s" % month_key)
        
        last_index = dataframe_period_end.index[-1]
        
#         print ("last index:%s " % last_index)
        
        if (month_key != last_index):
            temp_df = dataframe_subperiod.loc[dataframe_subperiod['Date']<end].head(n)
#             print ("\n")
#             print (temp_df.head(5))
            temp_df = temp_df.drop(['Date'], axis=1)
            
            empty_dict[month_key]= temp_df.values.flatten()
            count +=1
        else:
            pass
       
    return empty_dict
    
#get dictionary of the period end dates with the 30days previous features minus date as a flattened array of 360 entries
#reshape 30,12 will give the original dataframe config
MSFT_30days_flattened= get_ndays_monthly_flattened(reversed_df_MSFT,MSFT_df,30)
AAPL_30days_flattened = get_ndays_monthly_flattened(reversed_df_AAPL,AAPL_df,30)
AMZN_30days_flattened = get_ndays_monthly_flattened(reversed_df_AMZN,AMZN_df,30)


#create flattened dataframe objects for the dataframes with no feature engineering
MSFT_30days_flattened_no_fe = get_ndays_monthly_flattened(reversed_df_MSFT,MSFT_df_no_fe,30)
AAPL_30days_flattened_no_fe = get_ndays_monthly_flattened(reversed_df_AAPL,AAPL_df_no_fe,30)
AMZN_30days_flattened_no_fe= get_ndays_monthly_flattened(reversed_df_AMZN,AMZN_df_no_fe,30)







# print (list(MSFT_30days_flattened.keys())[0])
# print (list(MSFT_30days_flattened.values())[:1])
# print ("\n")
# print (list(AAPL_30days_flattened.values())[:1])
# print ("\n")
# print (list(AMZN_30days_flattened.values())[:1])


NameError: name 'MSFT_df_no_fe' is not defined

In [19]:
#assign the labels to pandas dataframe objects and give label title
MSFT_labels_df = pd.Series(MSFT_labels).to_frame("Label")
AAPL_labels_df= pd.Series(AAPL_labels).to_frame("Label")
AMZN_labels_df= pd.Series(AMZN_labels).to_frame("Label")

# print (MSFT_labels_df.head(10))
# print("\n")
# print(AAPL_labels_df.head(10))
# print("\n")
# print(AMZN_labels_df.head(10))


# Dictionary objects as JSON file

# Read Label Dataframes into CSV

In [20]:
export_MSFT= MSFT_labels_df.to_csv(r'MSFT_labels.csv')
export_AMZN = AMZN_labels_df.to_csv(r'AMZN_labels.csv') 
export_AAPL = AAPL_labels_df.to_csv(r'AAPL_labels.csv')

# Create flattened DataFrame with label

In [21]:
#no longer required to reverse the order of the labels dataframes



#convert the flattened dictionaries to dataframes 
flattened_df_MSFT = pd.DataFrame.from_dict(MSFT_30days_flattened).T
flattened_df_AAPL = pd.DataFrame.from_dict(AAPL_30days_flattened).T
flattened_df_AMZN = pd.DataFrame.from_dict(AMZN_30days_flattened).T

flattened_df_MSFT_no_fe = pd.DataFrame.from_dict(MSFT_30days_flattened_no_fe).T
flattened_df_AAPL_no_fe = pd.DataFrame.from_dict(AAPL_30days_flattened_no_fe).T
flattened_df_AMZN_no_fe = pd.DataFrame.from_dict(AMZN_30days_flattened_no_fe).T

# print (flattened_df_MSFT.head(2))
# print (flattened_df_AAPL.head(2))
# print (flattened_df_AMZN.head(2))

# print ("\n")
# print (AAPL_labels_df.head(2))
# print ("\n")
# print (AMZN_labels_df.head(2))

In [22]:
#merge the dataframes on the index 'Date', and ensure that only the dates present in the flattened dataframe are carried over

MSFT_flat_df = flattened_df_MSFT.merge(MSFT_labels_df, left_index=True, right_index=True)
AAPL_flat_df = flattened_df_AAPL.merge(AAPL_labels_df, left_index=True, right_index=True)
AMZN_flat_df = flattened_df_AMZN.merge(AMZN_labels_df, left_index=True, right_index=True)


MSFT_flat_df_no_fe = flattened_df_MSFT_no_fe.merge(MSFT_labels_df, left_index=True, right_index=True)
AAPL_flat_df_no_fe = flattened_df_AAPL_no_fe.merge(AAPL_labels_df, left_index=True, right_index=True)
AMZN_flat_df_no_fe = flattened_df_AMZN_no_fe.merge(AMZN_labels_df, left_index=True, right_index=True)






In [23]:
# print(MSFT_flat_df.head(10))
# print("\n")
# print(AAPL_flat_df.head(10))
# print("\n")
# print(AMZN_flat_df.head(10))

print (MSFT_flat_df_no_fe.head(10))


                   0         1         2         3         4         5  \
1998-03-31  0.429339  0.425285  0.432434  0.426907  0.020233  0.416758   
1998-04-30  0.444974  0.443069  0.449092  0.441714  0.005799  0.452974   
1998-05-29  0.457188  0.460549  0.464632  0.457249  0.015414  0.469769   
1998-06-30  0.430866  0.429412  0.436288  0.431822  0.007196  0.413277   
1998-07-31  0.551484  0.558813  0.561785  0.560046  0.022155  0.531636   
1998-08-31  0.590815  0.594805  0.595972  0.596456  0.025066  0.598815   
1998-09-30  0.568279  0.570527  0.558677  0.546756  0.025059  0.582387   
1998-10-30  0.590021  0.597111  0.599826  0.592997  0.014853  0.597288   
1998-11-30  0.551118  0.557660  0.562966  0.555070  0.016410  0.547636   
1998-12-31  0.671308  0.682872  0.680259  0.685175  0.005320  0.656468   

                   6         7         8         9  ...       141       142  \
1998-03-31  0.421097  0.423856  0.423509  0.034510  ...  0.712430  0.714881   
1998-04-30  0.447621  0.451

# Export dataframe with flattened data and labels

In [86]:
export_MSFT= MSFT_flat_df.to_csv(r'MSFT_flat.csv')
export_AMZN = AMZN_flat_df.to_csv(r'AMZN_flat.csv') 
export_AAPL = AAPL_flat_df.to_csv(r'AAPL_flat.csv')

In [24]:
export_MSFT= MSFT_flat_df_no_fe.to_csv(r'MSFT_flat_no_fe.csv')
export_AMZN = AMZN_flat_df_no_fe.to_csv(r'AMZN_flat_no_fe.csv') 
export_AAPL = AAPL_flat_df_no_fe.to_csv(r'AAPL_flat_no_fe.csv')

# Labelling MultiClass

In [56]:
def labelling_multiclass(dataframe):
    """This generates labels for the dataframes we wish to predict price.
    A label of 1 suggests an increase in price between months while a label of 0 indicates
    a decrease in price. If the current price is greater than the previous price then a"""
    dataframe = dataframe.apply(pd.to_numeric)
    #introduce the absolute price change as a variable
    dataframe['ATR'] = ta.volatility.average_true_range(dataframe.High, dataframe.Low, dataframe.Close, n=14, 
                                                        fillna=True)
    
    #calculate the percentage change of ATR between months 
    dataframe['ATR_%_change'] = abs(dataframe['ATR'].pct_change())
    
    #calculate the average ATR pct change
    average_ATR_pct_change = dataframe['ATR_%_change'].mean()
    
    
    
    empty_dict = dict()
    prev_close =0
    counter = 0
    
    #print (dataframe.head(5))
    
    for index,cols in dataframe.iterrows():
        
        counter +=1
        
        current_close = cols['Close']
        ATR_price_change = cols['ATR_%_change']
        
        if (float(prev_close) <= float(current_close)) and (float(ATR_price_change) <= float(average_ATR_pct_change)):
            
            #signals an increase in price between last month and next month
            
            empty_dict[index] = 1
        # 2 represents an increase that was strong greater than the average ATR   
        elif (float(prev_close) <= float(current_close)) and (float(ATR_price_change) >= float(average_ATR_pct_change)):
            empty_dict[index] = 2
        
        elif (float(prev_close) >= float(current_close)and (float(ATR_price_change) <= float(average_ATR_pct_change))):
            empty_dict[index]=0
            
        #3 represents a decrease in the price that had a larger ATR    
        elif (float(prev_close) >= float(current_close)and (float(ATR_price_change) >= float(average_ATR_pct_change))):
            empty_dict[index] = 3
        
        else:
            empty_dict[index]= 0
            

        #reassign prev close to the current close     
        prev_close = current_close
    
    return empty_dict

In [57]:
#dictionaries with dates as keys and values to correspond to strategies
MSFT_df_labels_multi= labelling_multiclass(reversed_df_MSFT)
AAPL_df_labels_multi = labelling_multiclass(reversed_df_AAPL)
AMZN_df_labels_multi = labelling_multiclass(reversed_df_AMZN)

In [58]:
#reversed dataframe for the multiclass approach
reversed_df_MSFT_multiclass= MSFT_df_monthly.iloc[::-1]
reversed_df_AAPL_multiclass= AAPL_df_monthly.iloc[::-1]
reversed_df_AMZN_multiclass = AMZN_df_monthly.iloc[::-1]

reversed_df_MSFT_multiclass.shape

(240, 5)

In [66]:
(AAPL_df_labels_multi)

{'1999-09-30': 0,
 '1999-10-29': 2,
 '1999-11-30': 2,
 '1999-12-31': 2,
 '2000-01-31': 2,
 '2000-02-29': 2,
 '2000-03-31': 2,
 '2000-04-28': 3,
 '2000-05-31': 3,
 '2000-06-30': 2,
 '2000-07-31': 3,
 '2000-08-31': 2,
 '2000-09-29': 0,
 '2000-10-31': 3,
 '2000-11-30': 3,
 '2000-12-29': 3,
 '2001-01-31': 2,
 '2001-02-28': 3,
 '2001-03-30': 2,
 '2001-04-30': 2,
 '2001-05-31': 3,
 '2001-06-29': 2,
 '2001-07-31': 3,
 '2001-08-31': 3,
 '2001-09-28': 3,
 '2001-10-31': 2,
 '2001-11-30': 2,
 '2001-12-31': 2,
 '2002-01-31': 2,
 '2002-02-28': 3,
 '2002-03-28': 2,
 '2002-04-30': 2,
 '2002-05-31': 3,
 '2002-06-28': 0,
 '2002-07-31': 3,
 '2002-08-30': 3,
 '2002-09-30': 3,
 '2002-10-31': 2,
 '2002-11-29': 3,
 '2002-12-31': 3,
 '2003-01-31': 2,
 '2003-02-28': 2,
 '2003-03-31': 3,
 '2003-04-30': 2,
 '2003-05-30': 1,
 '2003-06-30': 2,
 '2003-07-31': 2,
 '2003-08-29': 1,
 '2003-09-30': 3,
 '2003-10-31': 1,
 '2003-11-28': 0,
 '2003-12-31': 2,
 '2004-01-30': 1,
 '2004-02-27': 2,
 '2004-03-31': 1,
 '2004-04-

In [60]:
#Add the labels to the reversed DataFrames
reversed_df_MSFT_multiclass['Labels'] = MSFT_df_labels_multi.values()
reversed_df_AAPL_multiclass['Labels'] = AAPL_df_labels_multi.values()
reversed_df_AMZN_multiclass['Labels']= AMZN_df_labels_multi.values()

/Users/fahad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/fahad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/fahad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [61]:
reversed_df_MSFT_multiclass

,Open,High,Low,Close,Volume,Labels
Date,,,,,,
1999-09-30,46.1563,48.9375,44.4063,45.2813,977273600,0
1999-10-29,45.0938,47.6250,42.5313,46.2813,1062304400,1
1999-11-30,46.6250,47.2500,42.1875,45.5234,1465256000,0
1999-12-31,45.5313,59.9688,45.4375,58.3750,1260977800,2
2000-01-31,58.6875,59.3125,47.4375,48.9375,1274875200,3
2000-02-29,49.2500,55.0000,44.0625,44.6875,1334487600,3
2000-03-31,44.8125,57.5000,44.4688,53.1250,2028187600,2
2000-04-28,47.2188,48.2500,32.5000,34.8750,2258146600,3
2000-05-31,36.4375,37.0000,30.1875,31.2813,1344430800,0


In [64]:
MSFT_30days_flattened_multi= get_ndays_monthly_flattened(reversed_df_MSFT_multiclass,MSFT_df,30)
AAPL_30days_flattened_multi= get_ndays_monthly_flattened(reversed_df_AAPL_multiclass,AAPL_df,30)
AMZN_30days_flattened_multi= get_ndays_monthly_flattened(reversed_df_AMZN_multiclass,AMZN_df,30)

In [67]:
#assign the labels to pandas dataframe objects and give label title
MSFT_labels_df_multi = pd.Series(MSFT_df_labels_multi).to_frame("Label")
AAPL_labels_df_multi= pd.Series(AAPL_df_labels_multi).to_frame("Label")
AMZN_labels_df_multi= pd.Series(AMZN_df_labels_multi).to_frame("Label")

In [69]:
#convert the flattened dictionaries to dataframes 
flattened_df_MSFT_multi = pd.DataFrame.from_dict(MSFT_30days_flattened_multi).T
flattened_df_AAPL_multi= pd.DataFrame.from_dict(AAPL_30days_flattened_multi).T
flattened_df_AMZN_multi= pd.DataFrame.from_dict(AMZN_30days_flattened_multi).T

In [70]:
#merge the dataframes on the index 'Date', and ensure that only the dates present in the flattened dataframe are carried over

MSFT_flat_df_multi = flattened_df_MSFT_multi.merge(MSFT_labels_df_multi, left_index=True, right_index=True)
AAPL_flat_df_multi= flattened_df_AAPL_multi.merge(AAPL_labels_df_multi, left_index=True, right_index=True)
AMZN_flat_df_multi= flattened_df_AMZN_multi.merge(AMZN_labels_df_multi, left_index=True, right_index=True)




,0,1,2,3,4,5,6,7,8,9,...,351,352,353,354,355,356,357,358,359,Label
1999-10-29,0.466349,0.464312,0.461524,0.451180,0.026011,0.392614,0.315727,0.497359,0.602052,0.279564,...,0.423873,0.043776,0.320521,0.241663,0.503847,0.520134,0.322717,0.0,0.0,1
1999-11-30,0.456822,0.456725,0.462705,0.453426,0.054165,0.443036,0.393185,0.597602,0.625133,0.288069,...,0.493294,0.061996,0.657190,0.370815,0.506497,0.602052,0.306840,0.0,0.0,0
1999-12-31,0.457555,0.463948,0.463886,0.455367,0.037904,0.227714,0.237692,0.617219,0.610681,0.417673,...,0.441289,0.058201,0.373878,0.326769,0.556924,0.602052,0.294717,0.0,0.0,2
2000-01-31,0.627031,0.633163,0.635567,0.621822,0.013142,0.725520,0.368742,0.576235,0.545025,0.508448,...,0.423873,0.050973,0.187810,0.286367,0.640573,0.615368,0.350095,0.0,0.0,3
2000-02-29,0.506413,0.513656,0.512059,0.504278,0.043660,0.428842,0.278790,0.553748,0.515730,0.475369,...,0.597973,0.103595,0.472893,0.271999,0.569017,0.597827,0.441344,0.0,0.0,3
2000-03-31,0.458349,0.464312,0.455308,0.463681,0.058897,0.440447,0.205031,0.435280,0.476671,0.530836,...,0.589235,0.056587,0.707060,0.396004,0.550538,0.515730,0.535737,0.0,0.0,2
2000-04-28,0.555698,0.564457,0.544692,0.535348,0.048726,0.814838,0.488533,0.345891,0.385235,0.541510,...,0.512592,0.062481,0.685844,0.323278,0.512679,0.499325,0.493948,0.0,0.0,3
2000-05-31,0.319042,0.329692,0.326330,0.331695,0.060206,0.534000,0.360090,0.376361,0.391201,0.268996,...,0.486801,0.059892,0.442289,0.227743,0.352277,0.382821,0.587393,0.0,0.0,0
2000-06-30,0.288506,0.294368,0.292951,0.292676,0.052434,0.262965,0.261492,0.414780,0.409417,0.262073,...,0.368469,0.096032,0.594917,0.399152,0.351326,0.385235,0.384578,0.0,0.0,1
2000-07-31,0.385062,0.384317,0.386561,0.376479,0.026832,0.466565,0.314411,0.375597,0.327534,0.246631,...,0.309728,0.039759,0.545492,0.344459,0.413931,0.409630,0.265053,0.0,0.0,0


In [72]:
export_MSFT= MSFT_flat_df_multi.to_csv(r'MSFT_flat_multi.csv')
export_AMZN = AMZN_flat_df_multi.to_csv(r'AMZN_flat_multi.csv') 
export_AAPL = AAPL_flat_df_multi.to_csv(r'AAPL_flat_multi.csv')

In [120]:
#this custom encoder enables the json.dump() command to handle dataframes 
class JSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if hasattr(obj, 'to_json'):
            return obj.to_json(orient='records')
        return json.JSONEncoder.default(self, obj)

In [121]:
#dump the dictionaries into a json object which can be accessed easily in another notebook
with open('MSFT_30day.json', 'w') as fp:
    json.dump(MSFT_30day, fp, cls=JSONEncoder)
    
with open('AAPL_30day.json', 'w') as fp:
    json.dump(AAPL_30day, fp, cls=JSONEncoder)
    
with open('AMZN_30day.json', 'w') as fp:
    json.dump(AMZN_30day, fp, cls=JSONEncoder)